# Prototype
[*v0.1*]

Prototyping with TMDB.

# Setup

Cells in this section handle notebook setup, like importing packages and functions/vars from scripts.

## Imports

Import `stdlib` packages (i.e. `pathlib.Path`) and package dependencies.

### stdlib

In [1]:
from pathlib import Path
import json
from typing import Any, Optional, Union
import random

### Custom modules & vars

In [2]:
## Constants
from lib.constants import (
    auth_endpoint,
    movie_endpoint,
    token_endpoint,
    api_key,
    base_url,
    tv_endpoint,
    session_endpoint,
    popular_tv_endpoint,
    basic_auth_headers,
)

In [3]:
from utils.time_utils import benchmark
from utils.file_utils import check_file_exist

In [4]:
from core.config import api_settings, app_settings, logging_settings
from utils.logger import get_logger

In [5]:
from utils.tmdb_utils import (
    authenticate,
    get_popular_tv,
    get_request_token,
    retrieve_token,
    get_bad_ids,
    append_bad_id,
    check_bad_id,
    generate_rand_id,
    get_tv_episode,
)

In [6]:
from core.db import Base, create_base_metadata, get_engine, get_session
from domain.schemas.tmdb import tmdb_media_schemas, tmdb_responses

### Dependencies

Packages installed with `pip` (or some equivalent tool)

In [7]:
import httpx

## Global Vars

Variables for use throughout the notebook

### Notebook vars

In [8]:
nb_log: bool = True
nb_verbose: bool = False

### Object vars

In [9]:
## Get a logger
log = get_logger(__name__, level="INFO")

#### Database object vars

In [10]:
## Database engine/session
engine = get_engine(connection="db/demo.sqlite", echo=True)
SessionLocal = get_session(engine=engine)

## Functions

Notebook-level functions. These differ from functions imported from scripts in that they are either prototypes, or functions meant only for the notebook.

### Notebook Functions

In [11]:
## Create database metadata early in the script
create_base_metadata(base_obj=Base, engine=engine)

2023-06-19 20:54:37,698 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 20:54:37,699 INFO sqlalchemy.engine.Engine COMMIT


True

### Prototype

## Classes

Notebook-level classes. These differ from classes/models imported from scripts in that they are either prototypes, or functions meant only for the notebook.

# Operations

Functions & data operations.

In [12]:
## Test authentication
_auth = authenticate()
display(f"Auth response: [{_auth.status_code}: {_auth.reason_phrase}]")

[INFO][2023-06-19_20:54:37][utils.tmdb_utils][authenticate ln: 36]: Requesting https://api.themoviedb.org/3/authentication


'Auth response: [200: OK]'

In [13]:
## Test retrieving auth token
_token = get_request_token()
display(f"Token response: [{_token.status_code}: {_auth.reason_phrase}]")

## Retrieve token string from resonse
token = retrieve_token(_token.text)
display(f"Token ({type(token)}): {token}")

[INFO][2023-06-19_20:54:37][utils.tmdb_utils][get_request_token ln: 90]: Requesting https://api.themoviedb.org/3/authentication/token/new


'Token response: [200: OK]'

"Token (<class 'domain.schemas.tmdb.tmdb_responses.ReqToken'>): success=True expires_at='2023-06-20 01:54:38 UTC' request_token='5121a2c3b538171adb9524f1437bb7d5700ca789'"

## Examples

### Example responses

Load example responses from the TMDB API for analysis.

In [14]:
## Setup examples dir
examples_dir: str = "examples"
example_responses_dir: str = "responses"
ex_res_path: str = f"{examples_dir}/{example_responses_dir}"

if not Path(ex_res_path).exists():
    display(f"Creating {ex_res_path}")
    Path(ex_res_path).mkdir(exist_ok=True, parents=True)

In [15]:
## Setup movie/tv example response JSON file paths
ex_movie_res_file: str = f"{ex_res_path}/ex_movie_response.json"
ex_tv_res_file: str = f"{ex_res_path}/ex_tvshow_response.json"

In [16]:
try:
    with open(ex_movie_res_file, "r+") as movie_res_file:
        ex_movie_res: dict = json.loads(movie_res_file.read())

except Exception as exc:
    raise Exception(
        f"Unhandled exception opening file '{ex_movie_res_file}'. Details: {exc}"
    )

try:
    with open(ex_tv_res_file, "r+") as tv_res_file:
        ex_tv_res: dict = json.loads(tv_res_file.read())
except Exception as exc:
    raise Exception(
        f"Unhandled exception opening file '{ex_tv_res_file}'. Details: {exc}"
    )

In [17]:
ex_movie_keys = ex_movie_res.keys()
# display(f"Movie dict keys:")

# for k in ex_movie_keys:
#     display(f"Key [{k}] ({type(ex_movie_res[k]).__name__}): {ex_movie_res[k]}")

In [18]:
ex_tv_keys = ex_tv_res.keys()
# display(f"TV dict keys:")

# for k in ex_tv_keys:

In [19]:
shared_keys: list[str] = []
movie_only_keys: list[str] = []
tv_only_keys: list[str] = []

In [20]:
## Loop over example Movie response keys
for k in ex_movie_keys:
    if not k in ex_tv_keys:
        # display(f"Movie-specific key found: {k} ({type(ex_movie_res[k]).__name__})")
        movie_only_keys.append(k)

    else:
        if not k in shared_keys:
            shared_keys.append(k)

In [21]:
## Loop over example TV Show response keys
for k in ex_tv_keys:
    if not k in ex_movie_keys:
        # display(f"TV-specific key found: {k} ({type(ex_tv_res[k]).__name__})")

        tv_only_keys.append(k)

    else:
        if not k in shared_keys:
            shared_keys.append(k)

In [22]:
## Show keys that are specific to Movie responses
display(f"Keys only in movie response:")

for movie_k in movie_only_keys:
    display(movie_k)

'Keys only in movie response:'

'belongs_to_collection'

'budget'

'imdb_id'

'original_title'

'release_date'

'revenue'

'runtime'

'title'

'video'

In [23]:
## Show keys that are specific to TV Show responses
display(f"Keys only in tv response:")

for tv_k in tv_only_keys:
    display(tv_k)

'Keys only in tv response:'

'created_by'

'episode_run_time'

'first_air_date'

'in_production'

...

'number_of_seasons'

'origin_country'

'original_name'

'seasons'

'type'

In [24]:
## Display keys that appear in both Movie and TV Show responses
display(f"Keys in both movies and tv shows:")

display(shared_keys)

'Keys in both movies and tv shows:'

['adult',
 'backdrop_path',
 'genres',
 'homepage',
 'id',
 'original_language',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'spoken_languages',
 'status',
 'tagline',
 'vote_average',
 'vote_count']

### TV Show examples

In [25]:
## Retrieve popular TV shows
popular_tv = get_popular_tv()
display(
    f"Retrieve popular TV shows response: [{popular_tv.status_code}: {popular_tv.reason_phrase}]"
)

[INFO][2023-06-19_20:54:38][utils.tmdb_utils][get_popular_tv ln: 222]: Requesting https://api.themoviedb.org/3/tv/popular?language=en-US&page=1


'Retrieve popular TV shows response: [200: OK]'

In [26]:
## Create dictionary from popular_tv response
pop_tv_dict: dict = popular_tv.text_json()
display(pop_tv_dict.keys())

display(pop_tv_dict["results"][0])

display(f"genre_ids ({type(pop_tv_dict['results'][0]['genre_ids'])})")

## Create MediaResponse class instance from pop_tv_dict
pop_tv: tmdb_media_schemas.MediaResponse = tmdb_media_schemas.MediaResponse.parse_obj(
    pop_tv_dict
)

# display(f"Popular TV shows response type: ({type(pop_tv)}): {pop_tv}")

dict_keys(['page', 'results', 'total_pages', 'total_results'])

{'backdrop_path': '/t2rAdgjSh0WYbXzdOB5zTDqzdCI.jpg',
 'first_air_date': '2022-11-02',
 'genre_ids': [18],
 'id': 213713,
 'name': 'Faltu',
 'origin_country': ['IN'],
 'original_language': 'hi',
 'original_name': 'Faltu',
 'overview': "What's in a name? Amidst the arid landscape of Rajasthan, a young woman with dreamy eyes struggles to prove her worth.",
 'popularity': 2699.206,
 'poster_path': '/lgyFuoXs7GvKJN0mNm7z7OMOFuZ.jpg',
 'vote_average': 4.6,
 'vote_count': 29}

"genre_ids (<class 'list'>)"

In [27]:
## grab a random sample TV show from popular TV shows
pop_tv_results: int = len(pop_tv.results)
rand_pop_tv: int = random.randint(0, pop_tv_results - 1)

_sample: tmdb_media_schemas.MediaTVShow = pop_tv.results[rand_pop_tv]
display(f"Sample ({type(_sample)}): {_sample}")

"Sample (<class 'domain.schemas.tmdb.tmdb_media_schemas.MediaTVShow'>): adult=None backdrop_path='/hR2pFXMxVihldEk5WoSrV7Yq2ra.jpg' genres=None genre_ids=[9648, 18, 10765] overview='Unravel the mystery of a nightmarish town in middle America that traps all those who enter. As the unwilling residents fight to keep a sense of normalcy and search for a way out, they must also survive the threats of the surrounding forest – including the terrifying creatures that come out when the sun goes down.' popularity=1154.703 poster_path='/pnrv8tfOcWxu4CrB8N7xK0jYJsR.jpg' tmdb_id=124364 original_language='en' vote_average=8.3 vote_count=453 homepage=None production_companies=None runtime=None created_by=None episode_run_time=None first_air_date='2022-02-20' in_production=None languages=None last_air_date=None last_episode_to_air=None next_episode_to_air=None name='From' origin_country=['US'] original_name='From' networks=None number_of_episodes=None number_of_seasons=None seasons=None type=None"

In [28]:
## Retrieve full listing from TMDB
url = f"{api_settings.BASE_URL}/tv/{_sample.tmdb_id}"
display(f"Sample TV URL: {url}")

'Sample TV URL: https://api.themoviedb.org/3/tv/124364'

In [29]:
## Get full details of _sample TV show by doing a new TMDB lookup
_show = get_tv_episode(tmdb_id=_sample.tmdb_id)
_show_dict: dict = _show.text_json()

tv_show: tmdb_media_schemas.MediaTVShow = tmdb_media_schemas.MediaTVShow.parse_obj(
    _show_dict
)

[INFO][2023-06-19_20:54:38][utils.tmdb_utils][get_tv_episode ln: 274]: Requesting https://api.themoviedb.org/3/tv/124364


In [30]:
## Show sample TV show
display(f"Sample TV Show ({type(tv_show).__name__})")

display(tv_show)

'Sample TV Show (MediaTVShow)'

MediaTVShow(adult=False, backdrop_path='/hR2pFXMxVihldEk5WoSrV7Yq2ra.jpg', genres=[MediaGenres(id=9648, name='Mystery'), MediaGenres(id=18, name='Drama'), MediaGenres(id=10765, name='Sci-Fi & Fantasy')], genre_ids=None, overview='Unravel the mystery of a nightmarish town in middle America that traps all those who enter. As the unwilling residents fight to keep a sense of normalcy and search for a way out, they must also survive the threats of the surrounding forest – including the terrifying creatures that come out when the sun goes down.', popularity=1154.703, poster_path='/pnrv8tfOcWxu4CrB8N7xK0jYJsR.jpg', tmdb_id=124364, original_language='en', vote_average=8.293, vote_count=458, homepage='https://www.epix.com/series/from', production_companies=[ProductionCompanies(id=106544, logo_path='/psd84iF7PTGrKf4yFOStKj8JbAh.png', name='AGBO', origin_country='US'), ProductionCompanies(id=51593, logo_path=None, name='Midnight Radio', origin_country='US'), ProductionCompanies(id=6805, logo_path

### Random movie

In [31]:
## Lookup a movie using a randomly generated ID.
#  If ID is invalid, it will be added to the bad_movie_ids file
movie_objs: list[tmdb_media_schemas.MediaMovie] = []
rand_movie = generate_rand_id(type="movie")

display(f"Random movie ID: {rand_movie}")

'Random movie ID: 168'

In [32]:
url = f"https://api.themoviedb.org/3/movie/{rand_movie}?language=en-US"
display(f"URL: {url}")

'URL: https://api.themoviedb.org/3/movie/168?language=en-US'

In [33]:
## Get random movie
try:
    with httpx.Client(headers=basic_auth_headers) as client:
        res = client.get(url)

        if not res.status_code == 200:
            display(
                f"Non-200 response returned: [{res.status_code}: {res.reason_phrase}]: {res.text}"
            )

            display(f"Adding ID {rand_movie} to list of known bad IDs")

            append_bad_id(bad_id=rand_movie, bad_id_file="bad_movie_ids")

        results: tmdb_responses.ReqResponse = tmdb_responses.ReqResponse.parse_obj(
            res.__dict__
        )

        res_json: dict = json.loads(res.text)

        _movie: tmdb_media_schemas.MediaMovie = tmdb_media_schemas.MediaMovie.parse_obj(
            res_json
        )

        display(f"Movie: {_movie}")

        movie_objs.append(_movie)

except Exception as exc:
    raise Exception(
        f"Unhandled exception requesting movie with ID: {rand_movie}. Details: {exc}"
    )

'Movie: adult=False backdrop_path=\'/wN3dgwkiWSLrMVukPQIeccv5EJ6.jpg\' genres=[MediaGenres(id=878, name=\'Science Fiction\'), MediaGenres(id=12, name=\'Adventure\')] genre_ids=None overview="It\'s the 23rd century, and a mysterious alien power is threatening Earth by evaporating the oceans and destroying the atmosphere. In a frantic attempt to save mankind, Kirk and his crew must time travel back to 1986 San Francisco where they find a world of punk, pizza and exact-change buses that are as alien as anything they\'ve ever encountered in the far reaches of the galaxy. A thrilling, action-packed Star Trek adventure!" popularity=28.032 poster_path=\'/xY5TzGXJOB3L9rhZ1MbbPyVlW5J.jpg\' tmdb_id=168 original_language=\'en\' vote_average=7.189 vote_count=1285 homepage=\'https://www.paramountmovies.com/movies/star-trek-iv-the-voyage-home\' production_companies=[ProductionCompanies(id=4, logo_path=\'/gz66EfNoYPqHTYI4q9UEN4CbHRc.png\', name=\'Paramount\', origin_country=\'US\')] runtime=119 belon

In [34]:
display(movie_objs)

[MediaMovie(adult=False, backdrop_path='/wN3dgwkiWSLrMVukPQIeccv5EJ6.jpg', genres=[MediaGenres(id=878, name='Science Fiction'), MediaGenres(id=12, name='Adventure')], genre_ids=None, overview="It's the 23rd century, and a mysterious alien power is threatening Earth by evaporating the oceans and destroying the atmosphere. In a frantic attempt to save mankind, Kirk and his crew must time travel back to 1986 San Francisco where they find a world of punk, pizza and exact-change buses that are as alien as anything they've ever encountered in the far reaches of the galaxy. A thrilling, action-packed Star Trek adventure!", popularity=28.032, poster_path='/xY5TzGXJOB3L9rhZ1MbbPyVlW5J.jpg', tmdb_id=168, original_language='en', vote_average=7.189, vote_count=1285, homepage='https://www.paramountmovies.com/movies/star-trek-iv-the-voyage-home', production_companies=[ProductionCompanies(id=4, logo_path='/gz66EfNoYPqHTYI4q9UEN4CbHRc.png', name='Paramount', origin_country='US')], runtime=119, belongs